##### <p> Samuel Wolfe <br> July 23rd, 2023 <br> MSBA 206 <br> DMBA Chapter 8 Homework 8.1 & 8.2 </p>

In [252]:
%matplotlib inline
from pathlib import Path
import pandas as pd
import requests
import io
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.naive_bayes import MultinomialNB, CategoricalNB
from sklearn.metrics import accuracy_score
from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier, KNeighborsRegressor
from dmba import regressionSummary, classificationSummary, liftChart, gainsChart
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error
import matplotlib.pylab as plt
import numpy as np

In [253]:
def readFile(url):
    download = requests.get(url).content
    # Reading the downloaded content and turning it into a pandas dataframe
    df = pd.read_csv(io.StringIO(download.decode('utf-8')))
    return df
def statslist(df):
    dfStats = pd.DataFrame({'Mean' : df.mean(numeric_only=True),
            'SD' : df.std(numeric_only=True),
            'Min' : df.min(),
            'Max' : df.max(),
            'Median' : df.median(numeric_only=True),
            })
    return dfStats
def categorize(df):
    for x in df:
        df[x] = df[x].astype('category')
    return df

# 8.1

#### Loading df
#### Note: With Python 3.11.4 pd.get_dummies() does not default return values as uint8, returns as bool. Have to change post.

In [254]:
dfBank = readFile("https://raw.githubusercontent.com/wolfesamk/MSBA-206/main/dmba/UniversalBank.csv")
dfBank['Education'] = dfBank['Education'].astype('category')
dfBank.columns = dfBank.columns.str.replace(' ', '_')
dfBank = pd.get_dummies(dfBank, prefix_sep='_')
dfBank['Education_1'] = dfBank['Education_1'].astype('uint8')
dfBank['Education_2'] = dfBank['Education_2'].astype('uint8')
dfBank['Education_3'] = dfBank['Education_3'].astype('uint8')
print(dfBank.shape)
dfBank.head(5)

(5000, 16)


,ID,Age,Experience,Income,ZIP_Code,Family,CCAvg,Mortgage,Personal_Loan,Securities_Account,CD_Account,Online,CreditCard,Education_1,Education_2,Education_3
0,1,25,1,49,91107,4,1.6,0,0,1,0,0,0,1,0,0
1,2,45,19,34,90089,3,1.5,0,0,1,0,0,0,1,0,0
2,3,39,15,11,94720,1,1.0,0,0,0,0,0,0,1,0,0
3,4,35,9,100,94112,1,2.7,0,0,0,0,0,0,0,1,0
4,5,35,8,45,91330,4,1.0,0,0,0,0,0,1,0,1,0


#### Creating training and validation splits.

In [255]:
bank_trainData, bank_validData = train_test_split(dfBank, test_size=0.4, random_state=1)
print(bank_trainData.shape, bank_validData.shape)

(3000, 16) (2000, 16)


## 8.1.a Pivot Table

In [256]:
bank_trainData_8_1_a = bank_trainData[['CreditCard', 'Personal_Loan','Online']]
bank_short_PT_8_1_a = bank_trainData_8_1_a.pivot_table(index=['CreditCard', 'Personal_Loan'],
                    columns=['Online'], aggfunc=len)
bank_short_PT_8_1_a

Online                      0     1
CreditCard Personal_Loan           
0          0              792  1117
           1               73   126
1          0              327   477
           1               39    49

## 8.1.b Probability of 1,1,1

In [257]:
prob111_S = f'P(Loan = 1 | CC = 1 | Online = 1): {(bank_short_PT_8_1_a.iat[3,1]) / (bank_short_PT_8_1_a.iat[2,1] + bank_short_PT_8_1_a.iat[3,1]):.2%}'
prob111_S

'P(Loan = 1 | CC = 1 | Online = 1): 9.32%'

## 8.1.c

In [258]:
bank_trainData_8_1_c_1 = bank_trainData[['Personal_Loan','Online']]
bank_short_PT_8_1_c_1 = bank_trainData_8_1_c_1.pivot_table(index=['Personal_Loan'],
                    columns=['Online'], aggfunc=len, margins=True)
bank_short_PT_8_1_c_1

Online,0,1,All
Personal_Loan,,,
0,1119,1594,2713
1,112,175,287
All,1231,1769,3000


In [259]:
bank_trainData_8_1_c_2 = bank_trainData[['CreditCard', 'Personal_Loan']]
bank_short_PT_8_1_c_2 = bank_trainData_8_1_c_2.pivot_table(index=['Personal_Loan'],
                    columns=['CreditCard'], aggfunc=len, margins=True)
bank_short_PT_8_1_c_2

CreditCard,0,1,All
Personal_Loan,,,
0,1909,804,2713
1,199,88,287
All,2108,892,3000


#### 8.1.d.i

In [260]:
P_CC1_Loan1_S = f'P(CC = 1 ∣ Loan = 1): {(bank_short_PT_8_1_c_2.iat[1,1]) / (bank_short_PT_8_1_c_2.iat[1,2]):.2%}'
P_CC1_Loan1 = (bank_short_PT_8_1_c_2.iat[1,1]) / (bank_short_PT_8_1_c_2.iat[1,2])
P_CC1_Loan1_S

'P(CC = 1 ∣ Loan = 1): 30.66%'

#### 8.1.d.ii

In [261]:
P_Online1_Loan1_S = f'P(Online = 1 ∣ Loan = 1): {(bank_short_PT_8_1_c_1.iat[1,1]) / (bank_short_PT_8_1_c_1.iat[1,2]):.2%}'
P_Online1_Loan1 = (bank_short_PT_8_1_c_1.iat[1,1]) / (bank_short_PT_8_1_c_1.iat[1,2])
P_Online1_Loan1_S

'P(Online = 1 ∣ Loan = 1): 60.98%'

#### 8.1.d.iii

In [262]:
P_Loan1_LoanAll_S = f'P(Loan = 1) (the proportion of loan acceptors): {(bank_short_PT_8_1_c_2.iat[1,2]) / (bank_short_PT_8_1_c_2.iat[2,2]):.2%}'
P_Loan1_LoanAll = (bank_short_PT_8_1_c_2.iat[1,2]) / (bank_short_PT_8_1_c_2.iat[2,2])
P_Loan1_LoanAll_S

'P(Loan = 1) (the proportion of loan acceptors): 9.57%'

#### 8.1.d.iv

In [263]:
P_CC1_Loan0_S = f'P(CC = 1 ∣ Loan = 0): {(bank_short_PT_8_1_c_2.iat[0,1]) / (bank_short_PT_8_1_c_2.iat[0,2]):.2%}'
P_CC1_Loan0 = (bank_short_PT_8_1_c_2.iat[0,1]) / (bank_short_PT_8_1_c_2.iat[0,2])
P_CC1_Loan0_S

'P(CC = 1 ∣ Loan = 0): 29.64%'

#### 8.1.d.v

In [264]:
P_Online1_Loan0_S = f'P(Online = 1 ∣ Loan = 0): {(bank_short_PT_8_1_c_1.iat[0,1]) / (bank_short_PT_8_1_c_1.iat[0,2]):.2%}'
P_Online1_Loan0 = (bank_short_PT_8_1_c_1.iat[0,1]) / (bank_short_PT_8_1_c_1.iat[0,2])
P_Online1_Loan0_S

'P(Online = 1 ∣ Loan = 0): 58.75%'

#### 8.1.d.vi

In [265]:
P_Loan0_S = f'P(Loan = 0): {(bank_short_PT_8_1_c_2.iat[0,2]) / (bank_short_PT_8_1_c_2.iat[2,2]):.2%}'
P_Loan0 = (bank_short_PT_8_1_c_2.iat[0,2]) / (bank_short_PT_8_1_c_2.iat[2,2])
P_Loan0_S

'P(Loan = 0): 90.43%'

## 8.1.e

In [266]:
P_Bayes_Loan1_CC1_Online1 = f'Naive Bayes P(Loan = 1 ∣ CC = 1, Online = 1): {(P_CC1_Loan1 * P_Online1_Loan1 * P_Loan1_LoanAll)/ ((P_CC1_Loan1 * P_Online1_Loan1 * P_Loan1_LoanAll) + (P_CC1_Loan0 * P_Online1_Loan0 * P_Loan0)):.2%}'
P_Bayes_Loan1_CC1_Online1

'Naive Bayes P(Loan = 1 ∣ CC = 1, Online = 1): 10.20%'

## 8.1.f
#### Looking at the raw values, the value generated from the first pivot table in 8.1.b is lower at 9.32% while in 8.1.e the naive Bayes probability is 10.20%. Knowing that Naive Bayes makes the assumption that all variables are independent (hence naive) I would say the first pivot table solution in 8.1.b is more accurate as it takes into consideration all the values.

## 8.1.e

#### Preparing data

In [267]:
classes = [0,1]
predictors = ['Online','CreditCard']
outcome = ['Personal_Loan']

dfBank = readFile("https://raw.githubusercontent.com/wolfesamk/MSBA-206/main/dmba/UniversalBank.csv")
dfBank['Education'] = dfBank['Education'].astype('category')
dfBank.columns = dfBank.columns.str.replace(' ', '_')
dfBank = pd.get_dummies(dfBank, prefix_sep='_', drop_first=False)
dfBank['Education_1'] = dfBank['Education_1'].astype('uint8')
dfBank['Education_2'] = dfBank['Education_2'].astype('uint8')
dfBank['Education_3'] = dfBank['Education_3'].astype('uint8')
dfBank = categorize(dfBank.drop(columns = ['ID','ZIP_Code']))


#### Binning variables

In [268]:
dfBank['Age'] = pd.qcut(dfBank['Age'], q=5, labels=[1,2,3,4,5],  precision=0)

In [269]:
dfBank['Experience'] = pd.qcut(dfBank['Experience'], q=10, labels=[1,2,3,4,5,6,7,8,9,10],  precision=0)

In [270]:
dfBank['Income'] = pd.qcut(dfBank['Income'], q=6, labels=[1,2,3,4,5,6],  precision=0)

In [271]:
dfBank['CCAvg'] = pd.qcut(dfBank['CCAvg'], q=8, labels=[1,2,3,4,5,6,7,8],  precision=0)

In [272]:
X_dfBank = dfBank[predictors]
Y_dfBank = (dfBank[outcome]).astype(int)
scaler = MinMaxScaler()
scaler.fit(X_dfBank)
X_train, X_valid, y_train, y_valid = train_test_split(X_dfBank, Y_dfBank, test_size=0.40, random_state=1)
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)

In [273]:
X_dfBank

,Online,CreditCard
0,0,0
1,0,0
2,0,0
3,0,0
4,0,1
...,...,...
4995,1,0
4996,1,0
4997,0,0
4998,1,0


#### Run Naive Bayes

In [274]:
bank_nb = MultinomialNB(alpha=1)
bank_nb.fit(X_train, y_train.Personal_Loan)

# predict probabilities
predProb_train = bank_nb.predict_proba(X_train)
predProb_valid = bank_nb.predict_proba(X_valid)

# predict class membership
y_train_pred = bank_nb.predict(X_train)
y_valid_pred = bank_nb.predict(X_valid)

In [275]:
predProb_valid

array([[0.90422182, 0.09577818],
       [0.90433333, 0.09566667],
       [0.90433333, 0.09566667],
       ...,
       [0.90427817, 0.09572183],
       [0.90438962, 0.09561038],
       [0.90438962, 0.09561038]])

In [276]:
y_valid_pred

array([0, 0, 0, ..., 0, 0, 0])

In [277]:
train_df, valid_df = train_test_split(dfBank, test_size=0.4, random_state=1)

pd.set_option('display.precision', 4)
# probability of flight status
print(train_df['Personal_Loan'].value_counts() / len(train_df))
print()

for predictor in predictors:
    # construct the frequency table
    df = train_df[['Personal_Loan', predictor]]
    freqTable = df.pivot_table(index='Personal_Loan', columns=predictor, aggfunc=len)

    # divide each row by the sum of the row to get conditional probabilities
    propTable = freqTable.apply(lambda x: x / sum(x), axis=1)
    print(propTable)
    print()
pd.reset_option('display.precision')

Personal_Loan
0    0.9043
1    0.0957
Name: count, dtype: float64

Online              0       1
Personal_Loan                
0              0.4125  0.5875
1              0.3902  0.6098

CreditCard          0       1
Personal_Loan                
0              0.7036  0.2964
1              0.6934  0.3066



In [278]:
# classify a specific loan by searching in the dataset
# for a loan with the same predictor values
df = pd.concat([pd.DataFrame({'actual': y_valid.Personal_Loan, 'predicted': y_valid_pred}),
                pd.DataFrame(predProb_valid, index=y_valid.index)], axis=1)
mask = ((bank_validData.Online == 1) & (bank_validData.CreditCard == 1) &
        (bank_validData.Personal_Loan == 1))
df[mask]

,actual,predicted,0,1
348,1,0,0.904278,0.095722
1524,1,0,0.904278,0.095722
1273,1,0,0.904278,0.095722
1038,1,0,0.904278,0.095722
4883,1,0,0.904278,0.095722
1050,1,0,0.904278,0.095722
2809,1,0,0.904278,0.095722
891,1,0,0.904278,0.095722
566,1,0,0.904278,0.095722
2707,1,0,0.904278,0.095722


## 8.1.G I believe, if my results are correct, the resulting probabilities show that the Naive Bayes probability generated in 8.1.e is roughly the same as the probability show above. I think this makes sense because we generated these results using a Naive Bayes method, so the probabilities should match.

# 8.2

#### Importing the data and checking the shape of the df while also checking the values of MAX_SEV_IR.

In [279]:
dfAA = readFile("https://raw.githubusercontent.com/wolfesamk/MSBA-206/main/dmba/accidentsFull.csv")
print(dfAA.shape)
dfAA.MAX_SEV_IR.value_counts()

(42183, 24)


MAX_SEV_IR
1    20996
0    20721
2      466
Name: count, dtype: int64

#### The INJURY Column is created below.

In [280]:
dfAA['INJURY'] = np.where((dfAA.MAX_SEV_IR == 1) | (dfAA.MAX_SEV_IR == 2),1,0)
print(dfAA.shape)
dfAA.INJURY.value_counts()

(42183, 25)


INJURY
1    21462
0    20721
Name: count, dtype: int64

## 8.2.a

In [281]:
pred8_2_a = f'P(Injury):{(dfAA.INJURY.value_counts()[1])/ (len(dfAA)):.2%}'
pred8_2_a


'P(Injury):50.88%'

#### Given the probability found above, 50.88%. The prediction should technically be if accident is reported it will have an injury.

## 8.2.b

#### Creating isolated dataframe for easier use.

In [282]:
dfAA_b = dfAA.head(12)[['INJURY','WEATHER_R','TRAF_CON_R']]
response = ['INJURY']
predictors = ['WEATHER_R','TRAF_CON_R']

## 8.2.b.i

In [283]:
dfAA_b_PT_i = dfAA_b.pivot_table(index=predictors,
                    columns=response, aggfunc=len, fill_value=0, margins=True)
dfAA_b_PT_i

INJURY                0  1  All
WEATHER_R TRAF_CON_R           
1         0           1  2    3
          1           1  0    1
          2           1  0    1
2         0           5  1    6
          1           1  0    1
All                   9  3   12

In [284]:
prob = []

## 8.2.b.ii

In [285]:
P_I1_W1_T0_S = f'Exact Bayes P(Injury=yes|WEATHER_R = 1 & TRAF_CON_R =0): {(((dfAA_b_PT_i.iat[0,1]) / (dfAA_b_PT_i.iat[0,2]))*((dfAA_b_PT_i.iat[0,2]) / (dfAA_b_PT_i.iat[5,2])))/((dfAA_b_PT_i.iat[0,2]) / (dfAA_b_PT_i.iat[5,2])):.2%}'
P_I1_W1_T0 = (((dfAA_b_PT_i.iat[0,1]) / (dfAA_b_PT_i.iat[0,2]))*((dfAA_b_PT_i.iat[0,2]) / (dfAA_b_PT_i.iat[5,2])))/((dfAA_b_PT_i.iat[0,2]) / (dfAA_b_PT_i.iat[5,2]))
prob.append(P_I1_W1_T0)
P_I1_W1_T0_S

'Exact Bayes P(Injury=yes|WEATHER_R = 1 & TRAF_CON_R =0): 66.67%'

In [286]:
P_I1_W1_T1_S = f'Exact Bayes P(Injury=yes|WEATHER_R = 1 & TRAF_CON_R = 1): {(((dfAA_b_PT_i.iat[1,1]) / (dfAA_b_PT_i.iat[1,2]))*((dfAA_b_PT_i.iat[1,2]) / (dfAA_b_PT_i.iat[5,2])))/((dfAA_b_PT_i.iat[1,2]) / (dfAA_b_PT_i.iat[5,2])):.2%}'
P_I1_W1_T1 = (((dfAA_b_PT_i.iat[1,1]) / (dfAA_b_PT_i.iat[1,2]))*((dfAA_b_PT_i.iat[1,2]) / (dfAA_b_PT_i.iat[5,2])))/((dfAA_b_PT_i.iat[1,2]) / (dfAA_b_PT_i.iat[5,2]))
prob.append(P_I1_W1_T1)
P_I1_W1_T1_S

'Exact Bayes P(Injury=yes|WEATHER_R = 1 & TRAF_CON_R = 1): 0.00%'

In [287]:
P_I1_W1_T2_S = f'Exact Bayes P(Injury=yes|WEATHER_R = 1 & TRAF_CON_R = 2): {(((dfAA_b_PT_i.iat[2,1]) / (dfAA_b_PT_i.iat[2,2]))*((dfAA_b_PT_i.iat[2,2]) / (dfAA_b_PT_i.iat[5,2])))/((dfAA_b_PT_i.iat[2,2]) / (dfAA_b_PT_i.iat[5,2])):.2%}'
P_I1_W1_T2 = (((dfAA_b_PT_i.iat[2,1]) / (dfAA_b_PT_i.iat[2,2]))*((dfAA_b_PT_i.iat[2,2]) / (dfAA_b_PT_i.iat[5,2])))/((dfAA_b_PT_i.iat[2,2]) / (dfAA_b_PT_i.iat[5,2]))
prob.append(P_I1_W1_T2)
P_I1_W1_T2_S

'Exact Bayes P(Injury=yes|WEATHER_R = 1 & TRAF_CON_R = 2): 0.00%'

In [288]:
P_I1_W2_T0_S = f'Exact Bayes P(Injury=yes|WEATHER_R = 2 & TRAF_CON_R = 0): {(((dfAA_b_PT_i.iat[3,1]) / (dfAA_b_PT_i.iat[3,2]))*((dfAA_b_PT_i.iat[3,2]) / (dfAA_b_PT_i.iat[5,2])))/((dfAA_b_PT_i.iat[3,2]) / (dfAA_b_PT_i.iat[5,2])):.2%}'
P_I1_W2_T0 = (((dfAA_b_PT_i.iat[3,1]) / (dfAA_b_PT_i.iat[3,2]))*((dfAA_b_PT_i.iat[3,2]) / (dfAA_b_PT_i.iat[5,2])))/((dfAA_b_PT_i.iat[3,2]) / (dfAA_b_PT_i.iat[5,2]))
prob.append(P_I1_W2_T0)
P_I1_W2_T0_S

'Exact Bayes P(Injury=yes|WEATHER_R = 2 & TRAF_CON_R = 0): 16.67%'

In [289]:
P_I1_W2_T0_S = f'Exact Bayes P(Injury=yes|WEATHER_R = 2 & TRAF_CON_R = 1): {(((dfAA_b_PT_i.iat[4,1]) / (dfAA_b_PT_i.iat[4,2]))*((dfAA_b_PT_i.iat[4,2]) / (dfAA_b_PT_i.iat[5,2])))/((dfAA_b_PT_i.iat[4,2]) / (dfAA_b_PT_i.iat[5,2])):.2%}'
P_I1_W2_T0 = (((dfAA_b_PT_i.iat[4,1]) / (dfAA_b_PT_i.iat[4,2]))*((dfAA_b_PT_i.iat[4,2]) / (dfAA_b_PT_i.iat[5,2])))/((dfAA_b_PT_i.iat[4,2]) / (dfAA_b_PT_i.iat[5,2]))
prob.append(P_I1_W2_T0)
P_I1_W2_T0_S

'Exact Bayes P(Injury=yes|WEATHER_R = 2 & TRAF_CON_R = 1): 0.00%'

#### Exact Bayes P(Injury=yes|WEATHER_R = 2 & TRAF_CON_R = 2): This is a 0/0 situation and does not become valid.

## 8.2.b.iii

In [290]:
conditions = [((dfAA_b.WEATHER_R == 1) & (dfAA_b.TRAF_CON_R == 0)),
    ((dfAA_b.WEATHER_R == 1) & (dfAA_b.TRAF_CON_R == 1)),
    ((dfAA_b.WEATHER_R == 1) & (dfAA_b.TRAF_CON_R == 2)),
    ((dfAA_b.WEATHER_R == 2) & (dfAA_b.TRAF_CON_R == 0)),
    ((dfAA_b.WEATHER_R == 2) & (dfAA_b.TRAF_CON_R == 1))]

In [291]:
dfAA_b['PROB_INJURY'] = np.select(conditions, prob)
dfAA_b['INJURY_PRED'] = np.where(dfAA_b.PROB_INJURY > 0.5,1,0)
dfAA_b

,INJURY,WEATHER_R,TRAF_CON_R,PROB_INJURY,INJURY_PRED
0,1,1,0,0.666667,1
1,0,2,0,0.166667,0
2,0,2,1,0.000000,0
3,0,1,1,0.000000,0
4,0,1,0,0.666667,1
5,1,2,0,0.166667,0
6,0,2,0,0.166667,0
7,1,1,0,0.666667,1
8,0,2,0,0.166667,0
9,0,2,0,0.166667,0


## 8.2.b.iv

In [292]:
dfAA_b_PT_i

INJURY                0  1  All
WEATHER_R TRAF_CON_R           
1         0           1  2    3
          1           1  0    1
          2           1  0    1
2         0           5  1    6
          1           1  0    1
All                   9  3   12

In [293]:
P_I1_W1_T1_NB_S = f'Naive Bayes P(Injury=yes|WEATHER_R = 1 & TRAF_CON_R = 1): {((dfAA_b_PT_i.iat[0,1])/(dfAA_b_PT_i.iat[0,2])) * ((dfAA_b_PT_i.iat[1,1])/(dfAA_b_PT_i.iat[5,1])) * ((dfAA_b_PT_i.iat[0,2])/(dfAA_b_PT_i.iat[5,2])):.2%}'
P_I1_W1_T1_NB = ((dfAA_b_PT_i.iat[0,1])/(dfAA_b_PT_i.iat[0,2])) * ((dfAA_b_PT_i.iat[1,1])/(dfAA_b_PT_i.iat[5,1])) * ((dfAA_b_PT_i.iat[0,2])/(dfAA_b_PT_i.iat[5,2]))
P_I1_W1_T1_NB_S

'Naive Bayes P(Injury=yes|WEATHER_R = 1 & TRAF_CON_R = 1): 0.00%'

## 8.2.b.v

In [294]:
dfAA_b = dfAA.head(12)[['INJURY','WEATHER_R','TRAF_CON_R']]


In [295]:
dfAA_b['INJURY']

0     1
1     0
2     0
3     0
4     0
5     1
6     0
7     1
8     0
9     0
10    0
11    0
Name: INJURY, dtype: int32

In [296]:
X_AA_v = dfAA_b[predictors]
Y_AA_v = dfAA_b[response]
AA_nb = MultinomialNB(alpha=1)
AA_nb.fit(X_AA_v, Y_AA_v.INJURY)

# predict probabilities
predProb_AA_Bv = AA_nb.predict_proba(X_AA_v)

# predict class membership
y_predAA_Bv = AA_nb.predict(X_AA_v)

In [297]:
predProb_AA_Bv

array([[0.73282443, 0.26717557],
       [0.71491738, 0.28508262],
       [0.78177885, 0.21822115],
       [0.7966805 , 0.2033195 ],
       [0.73282443, 0.26717557],
       [0.71491738, 0.28508262],
       [0.71491738, 0.28508262],
       [0.73282443, 0.26717557],
       [0.71491738, 0.28508262],
       [0.71491738, 0.28508262],
       [0.71491738, 0.28508262],
       [0.84843129, 0.15156871]])

In [298]:
y_predAA_Bv

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [299]:
df = pd.concat([pd.DataFrame({'actual': Y_AA_v.INJURY, 'predicted': y_predAA_Bv}),
                pd.DataFrame(predProb_AA_Bv, index=Y_AA_v.index)], axis=1)
df

,actual,predicted,0,1
0,1,0,0.732824,0.267176
1,0,0,0.714917,0.285083
2,0,0,0.781779,0.218221
3,0,0,0.796680,0.203320
4,0,0,0.732824,0.267176
5,1,0,0.714917,0.285083
6,0,0,0.714917,0.285083
7,1,0,0.732824,0.267176
8,0,0,0.714917,0.285083
9,0,0,0.714917,0.285083


## 8.2.b.v Comparing this output with the one generated in 8.2.b.iii we see that in the earlier calculation we got 2/3 accuracy while in this model we got 0/3 correct. This is valid.

# 8.2.c

#### 8.2.c.i Looking at the categories, given we cannot use information gained after the accident to predict if an injury is going to happen, we have to look at conditions we can establish globally. This would be 'HOUR_I_R','ALIGN_I','WRK_ZONE','WKDY_I_R','INT_HWY','LGTCON_I_R','PROFIL_I_R','SPD_LIM','SUR_CON','TRAF_CON_R','TRAF_WAY','WEATHER_R' as these are known with simply the report of the accident having occurred at that location.

## 8.2.c.ii

#### preparing the data

In [300]:
predictors = ['HOUR_I_R','ALIGN_I','WRK_ZONE','WKDY_I_R','INT_HWY','LGTCON_I_R','PROFIL_I_R','SPD_LIM','SUR_COND','TRAF_CON_R','TRAF_WAY','WEATHER_R']
outcome = ['INJURY']

In [301]:
X_dfAA = pd.get_dummies(dfAA[predictors], prefix_sep='_', drop_first=False)
Y_dfAA = (dfAA[outcome]).astype(int)

scaler = MinMaxScaler()
scaler.fit(X_dfBank)
X_train, X_valid, y_train, y_valid = train_test_split(X_dfAA, Y_dfAA, test_size=0.40, random_state=1)
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)

In [302]:
# run naive Bayes
AA_nb = MultinomialNB(alpha=1)
AA_nb.fit(X_train, y_train.INJURY)

# predict probabilities
predProb_train = AA_nb.predict_proba(X_train)
predProb_valid = AA_nb.predict_proba(X_valid)

# predict class membership
y_train_pred = AA_nb.predict(X_train)
y_valid_pred = AA_nb.predict(X_valid)

In [303]:
print('Test Data Set')
classificationSummary(y_train, y_train_pred, class_names=classes) 
print()
print('Validation Data Set')
classificationSummary(y_valid, y_valid_pred, class_names=classes) 

Test Data Set
Confusion Matrix (Accuracy 0.5245)

       Prediction
Actual     0     1
     0  2197 10195
     1  1839 11078

Validation Data Set
Confusion Matrix (Accuracy 0.5221)

       Prediction
Actual    0    1
     0 1476 6853
     1 1211 7334


## 8.2.c.iii The overall error for the validation set is 47.79%

## 8.2.c.iv The percent improvement relative to the naive rule when using the validation set is 0.2399%

## 8.2.c.v

#### All the predictors

In [309]:
train_df, valid_df = train_test_split(dfAA, test_size=0.4, random_state=1)

pd.set_option('display.precision', 4)
pd.set_option('display.width', 250)
# probability of flight status
print(dfAA[response].value_counts() / len(train_df))
print()

for predictor in predictors:
    # construct the frequency table
    df = dfAA[['INJURY', predictor]]
    freqTable = df.pivot_table(index=response, columns=predictor, aggfunc=len)

    # divide each row by the sum of the row to get conditional probabilities
    propTable = freqTable.apply(lambda x: x / sum(x), axis=1)
    print(propTable)
    print()
pd.reset_option('display.precision')
pd.reset_option('display.width')

INJURY
1         0.8480
0         0.8187
Name: count, dtype: float64

HOUR_I_R       0       1
INJURY                  
0         0.5678  0.4322
1         0.5734  0.4266

ALIGN_I       1       2
INJURY                 
0        0.8707  0.1293
1        0.8663  0.1337

WRK_ZONE       0       1
INJURY                  
0         0.9760  0.0240
1         0.9788  0.0212

WKDY_I_R       0       1
INJURY                  
0         0.2177  0.7823
1         0.2387  0.7613

INT_HWY       0       1       9
INJURY                         
0        0.8498  0.1497  0.0005
1        0.8600  0.1392  0.0008

LGTCON_I_R       1       2       3
INJURY                            
0           0.6916  0.1254  0.1830
1           0.6972  0.1120  0.1908

PROFIL_I_R       0       1
INJURY                    
0           0.7504  0.2496
1           0.7629  0.2371

SPD_LIM          5       10      15      20      25      30      35      40      45      50      55      60      65      70      75
INJURY             

#### Just SPD_LIM

In [310]:
pd.set_option('display.precision', 4)
pd.set_option('display.width', 250)
# probability of flight status
print(dfAA[response].value_counts() / len(dfAA))
print()
# construct the frequency table
df = dfAA[['INJURY', 'SPD_LIM']]
freqTable = df.pivot_table(index=response, columns=['SPD_LIM'], aggfunc=len)

# divide each row by the sum of the row to get conditional probabilities
propTable = freqTable.apply(lambda x: x / sum(x), axis=1)
print(propTable)
print()
    
pd.reset_option('display.precision')
pd.reset_option('display.width')

INJURY
1         0.5088
0         0.4912
Name: count, dtype: float64

SPD_LIM          5       10      15      20      25      30      35      40      45      50      55      60      65      70      75
INJURY                                                                                                                             
0        9.6520e-05  0.0005  0.0045  0.0077  0.1083  0.0872  0.1928  0.0955  0.1564  0.0407  0.1595  0.0351  0.0662  0.0395  0.0061
1        1.8638e-04  0.0005  0.0042  0.0043  0.0913  0.0889  0.2119  0.1084  0.1560  0.0383  0.1532  0.0434  0.0626  0.0296  0.0073



#### 8.2.c.v looking at P(INJURY = No j SPD_LIM = 5) We get probability of 9.6520e-05. This is near zero in terms of chances of happening, so in a practical sense that means an accident at 5mph should never happen, but because of the nature of probability where it can never truly equal 0 or 1, our function will always provide a number, even if its in scientific notation indicating its so small as to need 8 leading zeros. This is a result of all of the other variables that play into causing injuries for car accidents still playing a role in the chances of injury, even for a 5mph crash.